## Essentials of Ray Remote Functions
When you call a remote function, it immediately returns an ObjectRef (a future),
which is a reference to a remote object. Ray creates and executes a task in the
background on a separate worker process and writes the result when finished into the
original reference. You can then call ray.get on the ObjectRef to obtain the value.
Note that ray.get is a blocking method waiting for task execution to complete before
returning the result.

## Remote Objects in Ray
A remote object is just an object, which may be on another node. ObjectRefs are like
pointers or IDs to objects that you can use to get the value from, or status of, the
remote function. In addition to being created from remote function calls, you can
also create ObjectRefs explicitly by using the ray.put function.


The example converts the
iterator to a list before passing it to ray.get. You need to do this when calling
ray.get takes in a list of futures or an individual future.1
The function waits until it has all the objects so it can return the list in order <br/><br/>

As with regular Ray remote functions, it’s important to think
about the amount of work done inside each remote invocation.
For example, using ray.remote to compute factorials recursively
will be slower than doing it locally since the work inside each
function is small even though the overall work can be large. The
exact amount of time depends on how busy your cluster is, but as a
general rule, anything executed in under a few seconds without any
special resources is not worth scheduling remotely

## Remote Functions Lifecycle
The invoking Ray process (called the owner) of a remote function schedules the
execution of a submitted task and facilitates the resolution of the returned ObjectRef
to its underlying value if needed. <br/><br/>
On task submission, the owner waits for all dependencies (i.e., ObjectRef objects that
were passed as an argument to the task) to become available before scheduling. The
dependencies can be local or remote, and the owner considers the dependencies to be
ready as soon as they are available anywhere in the cluster. When the dependencies
are ready, the owner requests resources from the distributed scheduler to execute the
task. Once resources are available, the scheduler grants the request and responds with
the address of a worker that will execute the function.<br/><br/>
At this point, the owner sends the task specification over gRPC to the worker. After
executing the task, the worker stores the return values. If the return values are small
(less than 100 KiB by default), the worker returns the values inline directly to the
owner, which copies them to its in-process object store. If the return values are large,
the worker stores the objects in its local shared memory store and replies to the
owner, indicating that the objects are now in distributed memory. This allows the
owner to refer to the objects without having to fetch the objects to its local node.
When a task is submitted with an ObjectRef as its argument, the worker must resolve
its value before it can start executing the task.<br/><br/>
Tasks can end in an error. Ray distinguishes between two types of task errors:<br/><br/>
**Application-level**<br/>
In this scenario, the worker process is alive, but the task ends in an error (e.g., a
task that throws an IndexError in Python).<br/><br/>
**System-level**<br/>
In this scenario, the worker process dies unexpectedly (e.g., a process that seg‐
faults, or if the worker’s local Raylet dies).<br/><br/>
Tasks that fail because of application-level errors are never retried. The exception
is caught and stored as the return value of the task. Tasks that fail because of system-level errors may be automatically retried up to a specified number of attempts.

In [33]:
import ray
import time
import random


things = list(range(20))
things.sort(reverse=True)

In [20]:
@ray.remote
def remote_task(x):
 time.sleep(x)
 return x

As you recall, the example remote function sleeps based on the input argument.
Since the range is in ascending order, calling the remote function on it will result
in futures that are completed in order. To ensure that the futures won’t complete
in order, you will need to modify the list. One way you can do this is by calling
things.sort(reverse=True) prior to mapping your remote function over things <br/><br/>
To see the difference between using ray.get and ray.wait, you can write a function
that collects the values from your futures with some time delay on each object to
simulate business logic<br/><br/>
The first option, not using ray.wait, is a bit simpler and cleaner to read, as shown in
Example 3-2, but is not recommended for production use.

In [21]:

""" waits for all futures to complete in the order in which they were submitted. 
Wasteful when each remote future has different execution time. 
Hence, you'd be stuck atleast as long as the long running future.
"""
def in_order():
 # Make the futures
 futures = list(map(lambda x: remote_task.remote(x), things))
 values = ray.get(futures)
 for v in values:
    print(f" Completed {v}")
    time.sleep(1) # Business logic goes here
   
in_order()

 Completed 19
 Completed 18
 Completed 17
 Completed 16
 Completed 15
 Completed 14
 Completed 13
 Completed 12
 Completed 11
 Completed 10
 Completed 9
 Completed 8
 Completed 7
 Completed 6
 Completed 5
 Completed 4
 Completed 3
 Completed 2
 Completed 1
 Completed 0


The second option is a bit more complex, as shown in Example 3-3. This works by
calling ray.wait to find the next available future and iterating until all the futures
have been completed. ray.wait returns two lists, one of the object references for
completed tasks (of the size requested, which defaults to 1) and another list of the rest
of the object references.

In [24]:
"""performs lazy evaluation of futures in arbitrary order, 
which means it would wait until the completion of next future
 and pull results instead of waiting for all to complete, and
will iterate until all the futures have been resolved.
"""
def as_available():
 # Make the futures
 futures = list(map(lambda x: remote_task.remote(x), things))
 # While we still have pending futures
 while len(futures) > 0:
    ready_futures, rest_futures = ray.wait(futures)
    print(f"Ready {len(ready_futures)} rest {len(rest_futures)}")
    for id in ready_futures:
        print(f'completed value {id}, result {ray.get(id)}')
        time.sleep(1) # Business logic goes here
    # We just need to wait on the ones that are not yet available
    futures = rest_futures
as_available()

Ready 1 rest 19
completed value ObjectRef(e5b2fae6e42f6bbaffffffffffffffffffffffff0100000001000000), result 4
Ready 1 rest 18
completed value ObjectRef(96eec688321e7882ffffffffffffffffffffffff0100000001000000), result 5
Ready 1 rest 17
completed value ObjectRef(852304542258e48cffffffffffffffffffffffff0100000001000000), result 6
Ready 1 rest 16
completed value ObjectRef(74701ed4b46295a6ffffffffffffffffffffffff0100000001000000), result 3
Ready 1 rest 15
completed value ObjectRef(641105d6352e1cecffffffffffffffffffffffff0100000001000000), result 1
Ready 1 rest 14
completed value ObjectRef(f173ee62d1d97be7ffffffffffffffffffffffff0100000001000000), result 0
Ready 1 rest 13
completed value ObjectRef(b888db822d6cb570ffffffffffffffffffffffff0100000001000000), result 9
Ready 1 rest 12
completed value ObjectRef(7baaf72767944862ffffffffffffffffffffffff0100000001000000), result 8
Ready 1 rest 11
completed value ObjectRef(d0cc07993b31210dffffffffffffffffffffffff0100000001000000), result 11
Ready 1 r

Running these functions side by side with timeit.time, you can see the difference
in performance. It’s important to note that this performance improvement depends
on how long the nonparallelized business logic (the logic in the loop) takes. If you’re
just summing the results, using ray.get directly could be OK, but if you’re doing
something more complex, you should use ray.wait. When we run this, we see that
ray.wait performs roughly twice as fast. You can try varying the sleep times and see
how it works out.<br/><br/>
You may wish to specify one of the few optional parameters to ray.wait:<br/><br/>
**num_returns**<br/><br/>
The number of ObjectRef objects for Ray to wait for completion before return‐
ing. You should set num_returns to less than or equal to the length of the input
list of ObjectRef objects; otherwise, the function throws an exception.2 The
default value is 1.<br/><br/>
**timeout**<br/><br/>
The maximum amount of time in seconds to wait before returning. This defaults
to −1 (which is treated as infinite).<br/><br/>
**fetch_local**<br/><br/>
You can disable fetching of results by setting this to false if you are interested
only in ensuring that the futures are completed.

Ray’s get and wait functions handle timeouts slightly differently. Ray doesn’t raise an
exception on ray.wait when a timeout occurs; instead, it simply returns fewer ready
futures than num_returns. However, if ray.get encounters a timeout, Ray will raise a
GetTimeoutError. Note that the return of the wait/get function does not mean that
your remote function will be terminated; it will still run in the dedicated process. You
can explicitly terminate your future (see the following tip) if you want to release the
resources

Since ray.wait can return results in any order, it’s essential to not
depend on the order of the results. If you need to do different
processing with different records (e.g., test a mix of group A and
group B), you should encode this in the result (often with types)

If you have a task that does not finish in a reasonable time (e.g., a straggler), you can
cancel the task by using ray.cancel with the same ObjectRef used to wait/get. You
can modify the previous ray.wait example to add a timeout and cancel any “bad”
tasks, resulting in something like Example 3-4.

In [30]:
import threading
futures = list(map(lambda x: remote_task.remote(x), [1, threading.TIMEOUT_MAX]))
# While we still have pending futures
while len(futures) > 0:
 # In practice, 10 seconds is too short for most cases
 ready_futures, rest_futures = ray.wait(futures, timeout=10, num_returns=1)
 # If we get back anything less than num_returns 
 if len(ready_futures) < 1:
    print(f"Timed out on {rest_futures}")
    # Canceling is a good idea for long-running, unneeded tasks
    ray.cancel(*rest_futures)
    # You should break since you exceeded your timeout
    break
 for id in ready_futures:
    print(f'completed value {id}, result {ray.get(id)}')
    futures = rest_futures


completed value ObjectRef(ea19b1b85fbbce81ffffffffffffffffffffffff0100000001000000), result 1
Timed out on [ObjectRef(4abf1de3f84b576dffffffffffffffffffffffff0100000001000000)]


Canceling a task should not be part of your normal program flow.
If you find yourself having to frequently cancel tasks, you should
investigate what’s going on. Any subsequent calls to wait or get
for a canceled task are unspecified and could raise an exception or
return incorrect results.


Another minor point that we skipped in the previous chapter is that while the
examples so far return only a single value, Ray remote functions can return multiple
values, as with regular Python functions

Fault tolerance is an important consideration for those running in a distributed
environment. Say the worker executing the task dies unexpectedly (because either the
process crashed or the machine failed). Ray will rerun the task (after a delay) until
either the task succeeds or the maximum number of retries is exceeded. We cover
fault tolerance more in Chapter 5.

## Composition of Remote Ray Functions
You can make your remote functions even more powerful by composing them.
The two most common methods of composition with remote functions in Ray are
**pipelining** and **nested parallelism**. You can compose your functions with nested
parallelism to express recursive functions. Ray also allows you to express sequential
dependencies without having to block or collect the result in the driver, known as
pipelining.
You can build a pipelined function by using ObjectRef objects from an earlier
ray.remote as parameters for a new remote function call. Ray will automatically
fetch the ObjectRef objects and pass the underlying objects to your function. This
approach allows for easy coordination between the function invocations. Addition‐
ally, such an approach minimizes data transfer; the result will be sent directly to the
node where execution of the second remote function is executed. A simple example
of such a sequential calculation is presented in Example 3-5.

In [36]:
@ray.remote
def generate_number(s: int, limit: int, sl: float) -> int :
 random.seed(s)
 time.sleep(sl)
 return random.randint(0, limit)
@ray.remote
def sum_values(v1: int, v2: int, v3: int) -> int :
 return v1+v2+v3
# Get result
print(ray.get(sum_values.remote(generate_number.remote(1, 10, .1),
 generate_number.remote(5, 20, .2), generate_number.remote(7, 15, .3))))

31


This code defines two remote functions and then starts three instances of the first
one. ObjectRef objects for all three instances are then used as input for the second
function. In this case, Ray will wait for all three instances to complete before startingto execute sum_values. You can use this approach not only for passing data but
also for expressing basic workflow style dependencies. There is no restriction on the
number of ObjectRef objects you can pass, and you can also pass “normal” Python
objects at the same time.


You cannot use Python structures (for example, lists, dictionaries, or classes) contain‐
ing ObjectRef instead of using ObjectRef directly. Ray waits for and resolves only
ObjectRef objects that are passed directly to a function. If you attempt to pass a
structure, you will have to do your own ray.wait and ray.get inside the function.
Example 3-6 is a variation of Example 3-5 that does not work